In [1]:
"""Необходимо написать скрипт для парсинга диалогов из файла
test_data.csv. Получившийся скрипт необходимо выложить в гит
репозиторий и прислать ссылку в качестве результата прохождения
тестового задания. Данные выкладывать в гит не следует.
2. Главные задачи, которые должен выполнять скрипт:
a. Извлекать реплики с приветствием – где менеджер поздоровался.
b. Извлекать реплики, где менеджер представил себя.
c. Извлекать имя менеджера.
d. Извлекать название компании.
e. Извлекать реплики, где менеджер попрощался.
f. Проверять требование к менеджеру: «В каждом диалоге
обязательно необходимо поздороваться и попрощаться с
клиентом»
3. Рекомендации:
a. Сделать локальную копию файла test_data.csv, в исходнике
никакие данные не менять!
b. Можно создать дополнительное поле в таблице test_data.csv, куда
будет сохраняться результат парсинга – например, напротив
реплики в столбце “insight” можно ставить флаг того, что эта
реплика с приветствием greeting=True.
c. Для выполнения задачи можно использовать любые библиотеки и
NLP модели.
d. Попробуйте учесть возможные синонимичные выражения, которые
могут помочь с извлечением данных сущностей."""

'Необходимо написать скрипт для парсинга диалогов из файла\ntest_data.csv. Получившийся скрипт необходимо выложить в гит\nрепозиторий и прислать ссылку в качестве результата прохождения\nтестового задания. Данные выкладывать в гит не следует.\n2. Главные задачи, которые должен выполнять скрипт:\na. Извлекать реплики с приветствием – где менеджер поздоровался.\nb. Извлекать реплики, где менеджер представил себя.\nc. Извлекать имя менеджера.\nd. Извлекать название компании.\ne. Извлекать реплики, где менеджер попрощался.\nf. Проверять требование к менеджеру: «В каждом диалоге\nобязательно необходимо поздороваться и попрощаться с\nклиентом»\n3. Рекомендации:\na. Сделать локальную копию файла test_data.csv, в исходнике\nникакие данные не менять!\nb. Можно создать дополнительное поле в таблице test_data.csv, куда\nбудет сохраняться результат парсинга – например, напротив\nреплики в столбце “insight” можно ставить флаг того, что эта\nреплика с приветствием greeting=True.\nc. Для выполнения з

In [2]:
#import libraries
#for pymorphy, Russian version specifically is needed

import pandas as pd
import pymorphy2

morph = pymorphy2.MorphAnalyzer(lang='ru')


In [3]:
#open file
df = pd.read_csv('test_data.csv')

In [4]:
#useful scripts for later use
def get_company_name(r):
    if r in company_words:
        return company_words[r]
    else:
        return "НЕ ПРЕДСТАВИЛ КОМПАНИЮ"
def get_manager_name(r):
    if r in name_words:
        return name_words[r]
    else:
        return "НЕ ПРЕДСТАВИЛСЯ"

def introduction_test(r):
    if r in farewell_dict and r in introduction_dict:
        return 1
    else:
        return 0

In [5]:
#arrays for data on whether sentence is greeting, introduction, etc
is_greeting_list = []
is_introduction_list = []
is_farewell_list = []

name_words = dict()
company_words = dict()
introduction_dict = dict()
farewell_dict = dict()

for index, row in df.iterrows():
    line = row.text.lower()
    
    #"degree" means whether the sentence is at the start, middle or end of the dialogue. we only need the former and the latter
    degree = 0

    is_greeting = 0
    is_introduction = 0
    is_farewell = 0
    
    introductions = ["здравствуйте", "алло", "привет", "день добрый", "добрый день"]
    farewells = ["хорошего дня", "хорошего вечера", "доброй ночи", "до свидания"]
    
    #checking the position of the sentence
    if row.line_n < 10:
        degree = 1
        
    index_next = index + 10
    if index_next >= len(df):
        degree = -1
    elif df.iloc[index_next].line_n < 10:
        degree = -1
     
    #if sentence is at the start, check for words that could be introduction markers, if at the end, for farewell markers
    if degree == 1:
        if ("зовут" in line and "меня" in line or "мое" in line and "имя" in line) and row.role == "manager":
            is_introduction = 1
        if any(ext in line for ext in introductions) and row.role == "manager":
            is_greeting = 1
    if degree == -1:
        if any(ext in line for ext in farewells) and row.role == "manager":
            is_farewell = 1
    
    if is_farewell:
        farewell_dict[row.dlg_id] = 1
    if is_introduction:
        introduction_dict[row.dlg_id] = 1
        
        #checking for company name
        line_split = line.split()
        company_index = next((i for i in range(len(line_split)) if "компания" in line_split[i]), -1)
        
        if company_index != -1:
            company_name = []
            #take the words that follow the word "компания" and checking their morphology. 
            #if one of them is not a noun or adjective, probably new sentence, and the name of the company ends
            for i in range(company_index + 1, company_index + 4):
                if i >= len(line_split):
                    break
                else:
                    morph_parsed = str(morph.parse(line_split[i])[0][1]).split(',')
                    if morph_parsed[0] == 'ADJF' or morph_parsed[0] == 'NOUN':
                        company_name.append(line_split[i])
                    else:
                        break
            company_name = " ".join(company_name)
            company_words[row.dlg_id] = company_name
        name_indices = set()
        
        greetings = ["меня", "зовут", "мое", "имя"]
        #checking for manager name
        #similar logic, check neighbours, except in both directions
        #if one of the neighbours is a morphological name, we take it
        for greeting in greetings:
            greet_index = next((i for i in range(len(line_split)) if greeting in line_split[i]), -2)
            name_indices.add(greet_index + 1)
            name_indices.add(greet_index - 1)

        for i in name_indices:
            if i >= 0:
                morph_parsed = str(morph.parse(line_split[i])[0][1]).split(',')
                if morph_parsed[0] == 'NOUN':
                    if "Name" in morph_parsed[3]:
                        name_words[row.dlg_id] = line_split[i]
                        
    
    is_greeting_list.append(is_greeting)
    is_introduction_list.append(is_introduction)
    is_farewell_list.append(is_farewell)


In [6]:
#save data in dataframe
#is_greeting = whether this sentence is a greeting
#is_introduction = whether this sentence is an introduction
#is_farewell = whether this sentence is a farewell

#manager_name = manager name in this dialogue. if none found, warning is made
#company_name = company name in this dialogue. if none found, warning is made
#did_intro_fare = whether the manager both introduced himself and said goodbye in this dialogue (condition e)

df['is_greeting'] = is_greeting_list
df['is_introduction'] = is_introduction_list
df['is_farewell'] = is_farewell_list

df['manager_name'] = df['dlg_id'].apply(get_manager_name)
df['company_name'] = df['dlg_id'].apply(get_company_name)
df['did_intro_fare'] = df['dlg_id'].apply(introduction_test)

df.to_csv('test_data_analyzed.csv', index = False, index_label = False)

df

,dlg_id,line_n,role,text,is_greeting,is_introduction,is_farewell,manager_name,company_name,did_intro_fare
0,0,0,client,Алло,0,0,0,ангелина,диджитал бизнес,1
1,0,1,manager,Алло здравствуйте,1,0,0,ангелина,диджитал бизнес,1
2,0,2,client,Добрый день,0,0,0,ангелина,диджитал бизнес,1
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,0,1,0,ангелина,диджитал бизнес,1
4,0,4,client,Ага,0,0,0,ангелина,диджитал бизнес,1
...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,0,0,0,НЕ ПРЕДСТАВИЛСЯ,НЕ ПРЕДСТАВИЛ КОМПАНИЮ,0
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,0,0,0,НЕ ПРЕДСТАВИЛСЯ,НЕ ПРЕДСТАВИЛ КОМПАНИЮ,0
477,5,140,client,Спасибо спасибо,0,0,0,НЕ ПРЕДСТАВИЛСЯ,НЕ ПРЕДСТАВИЛ КОМПАНИЮ,0
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,0,0,0,НЕ ПРЕДСТАВИЛСЯ,НЕ ПРЕДСТАВИЛ КОМПАНИЮ,0
